### 데이터 로드

In [4]:
from tensorflow.keras.utils import get_file

ratings_train_path = get_file("ratings_train.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
ratings_test_path = get_file("ratings_test.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

ratings_train_path, ratings_test_path

('C:\\Users\\Playdata\\.keras\\datasets\\ratings_train.txt',
 'C:\\Users\\Playdata\\.keras\\datasets\\ratings_test.txt')

In [5]:
# 데이터프레임 생성
import pandas as pd

train_df = pd.read_csv(ratings_train_path, sep='\t')
test_df = pd.read_csv(ratings_test_path, sep='\t')

# print(train_df.head())
# print(test_df.head())

### 데이터 전처리

In [6]:
# 결측치 제거
print(train_df.isnull().sum())

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

print(train_df.isnull().sum())

id          0
document    5
label       0
dtype: int64
id          0
document    0
label       0
dtype: int64


In [7]:
import re
from konlpy.tag import Okt

# 한글 토큰화 전처리 (특수문자 처리, 어간 추출, 불용어 처리) -> 함수
tokenizer = Okt()

stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def preprocess(text):
    # 1. 특수문자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ0-9\s]', "", text)
    # 2. 형태소 분석 및 어간 추출
    tokens = tokenizer.morphs(text, stem=True)
    # 3. 불용어 제거
    tokens = [token for token in tokens if token not in stop_words]
    return tokens



In [8]:
# 훈련데이터 전처리
train_df['tokens'] = train_df['document'].apply(lambda x: preprocess(str(x)))


In [9]:
# 평가데이터 전처리
test_df['tokens'] = test_df['document'].apply(lambda x: preprocess(str(x)))

In [10]:
# !pip install torchtext==0.16.2

In [11]:
# !pip uninstall torch torchtext -y
# !pip install torch torchtext --index-url https://download.pytorch.org/whl/cpu

In [12]:
from torchtext.vocab import build_vocab_from_iterator

# 토큰 생성기 (전처리된 토큰을 하나씩 전달)
def yield_tokens(data): 
    for tokens in data:
        yield tokens

# 단어 사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_df['tokens']), specials=["<pad>", "<unk>"])

# '<unk>'는 미등록 단어, '<pad>'는 패딩에 사용
vocab.set_default_index(vocab["<unk>"])

print(f"단어 수: {len(vocab)}")
print(vocab.get_stoi())  # 단어 -> 인덱스 매핑 확인


단어 수: 45711
{'이유': 327, '온갓': 38305, '아저씨': 924, '안봣으': 18384, '히든싱어': 10772, '으로의': 5080, '에러': 3589, '딸': 696, '악당': 1396, '재방송': 3358, '랐슴': 31364, '지니': 7092, '드팔마': 11017, '난간': 20831, '그게': 855, 'ㅠㅡㅜ': 13059, '켄신': 16353, '향토': 44871, '무비': 1012, '자연도': 24040, '있엇으리': 39993, '열기': 13960, '씌여': 36725, '골빈': 8832, '젼혀': 24323, '마당': 7969, '<pad>': 0, '징끗하': 41776, '유병언': 39002, '여한': 37943, '이제': 309, '김남주': 11949, '조상': 9202, '뻘': 8493, '쪼꼬릿': 41907, '얼렁': 37644, '전두환': 7080, '이리': 553, '기뻣습니': 28458, '비노': 5185, '냐': 222, '바로바로': 13546, '네나': 20904, '쩝': 1506, '바쿠': 33188, '재미있다': 52, '은지': 12590, '만족감': 7604, '젠더': 19097, '삼류스멜': 35023, 'ㄱㄱㄱ': 9336, '당시': 360, '나답': 29031, '킵': 43292, '액': 5368, '다니다': 976, '못밑': 32625, '좀글': 41178, '인셉션': 4521, '못': 36, '엇슴': 18491, '작인게': 40133, '엉뚱하다': 3260, '신속하다': 22917, '아슬아슬하다': 10398, '피보나치수열': 44369, '새디즘': 35113, '닭털': 13328, '젖소': 19079, '든': 493, '투야': 43602, '<unk>': 1, '뇌': 2006, 'ㅜㅜ': 286, '가관': 3221, '뿩': 34779, '감동': 40, '봉황': 1

In [13]:
# 토큰을 시퀀스로 변환하는 함수
def text_to_sequence(tokens):
    return [vocab[token] for token in tokens]

# 훈련/평가 데이터 변환
train_df['sequence'] = train_df['tokens'].apply(text_to_sequence)
test_df['sequence'] = test_df['tokens'].apply(text_to_sequence)

print(train_df[['tokens', 'sequence']].head())


                                              tokens  \
0                             [아, 더빙, 진짜, 짜증나다, 목소리]   
1       [흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍다, 않다]   
2                     [너, 무재, 밓었, 다그, 래서, 보다, 추천, 다]   
3               [교도소, 이야기, 구먼, 솔직하다, 재미, 없다, 평점, 조정]   
4  [사이, 몬페, 그, 익살스럽다, 연기, 돋보이다, 영화, 스파이더맨, 에서, 늙다...   

                                            sequence  
0                            [53, 462, 17, 263, 670]  
1  [932, 465, 43, 609, 2, 218, 1477, 25, 986, 683...  
2         [391, 2501, 33124, 2355, 5652, 3, 225, 10]  
3            [6630, 110, 8334, 221, 58, 5, 26, 3729]  
4  [1051, 21728, 30, 9790, 25, 842, 2, 2629, 22, ...  


In [14]:
# padding 작업
import torch
from torch.nn.utils.rnn import pad_sequence

MAX_LEN = 30

def pad_sequences(sequences, max_len):
    return pad_sequence([torch.tensor(seq[:max_len]) for seq in sequences], batch_first=True)

train_padded = pad_sequences(train_df['sequence'], MAX_LEN)
test_padded = pad_sequences(test_df['sequence'], MAX_LEN)

print(train_padded.shape)

torch.Size([149995, 30])


### 모델 생성 및 학습

In [15]:
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = self.embedding(x)                # (batch_size, seq_len) → (batch_size, seq_len, embed_dim)
        _, (hidden, _) = self.lstm(x)        # hidden: (2, batch_size, hidden_dim)
        hidden = torch.cat((hidden[0], hidden[1]), dim=1)  # 양방향 결합
        return self.fc(hidden)               # 출력: 감정 예측 값


In [16]:
import torch
from torch.utils.data import Dataset

# 커스텀 Dataset 정의
class SentimentDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]


In [17]:
# 모델 인스턴스 생성
# 훈련 데이터와 라벨 준비
train_sequences = torch.tensor(train_padded, dtype=torch.long)
train_labels = torch.tensor(train_df['label'].values, dtype=torch.long)

# 평가 데이터와 라벨 준비
test_sequences = torch.tensor(test_padded, dtype=torch.long)
test_labels = torch.tensor(test_df['label'].values, dtype=torch.long)

# Dataset 객체 생성
train_dataset = SentimentDataset(train_sequences, train_labels)
test_dataset = SentimentDataset(test_sequences, test_labels)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_19836\1028312389.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_sequences = torch.tensor(train_padded, dtype=torch.long)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_19836\1028312389.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_sequences = torch.tensor(test_padded, dtype=torch.long)


In [18]:
from torch.utils.data import DataLoader

# 하이퍼파라미터 설정
BATCH_SIZE = 64

# DataLoader 생성 (shuffle=True로 데이터 섞기)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# DataLoader 확인
for batch in train_loader:
    sequences, labels = batch
    print("입력 데이터 형태:", sequences.shape)  # (64, 30) - 64개 샘플, 시퀀스 길이 30
    print("라벨 형태:", labels.shape)            # (64,)
    break


입력 데이터 형태: torch.Size([64, 30])
라벨 형태: torch.Size([64])


In [19]:
# Bidirectional LSTM 모델 정의
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        # 임베딩 층: (batch_size, seq_len) -> (batch_size, seq_len, embed_dim)
        x = self.embedding(x)

        # LSTM 층: hidden은 (2, batch_size, hidden_dim) 형태
        _, (hidden, _) = self.lstm(x)

        # 양방향의 hidden 상태를 결합 (forward + backward)
        hidden = torch.cat((hidden[0], hidden[1]), dim=1)

        # 완전 연결층을 통과한 결과 반환 (감정 예측 값)
        return self.fc(hidden)


In [20]:
# 하이퍼파라미터 설정
vocab_size = len(vocab)  # 단어 사전 크기
embed_dim = 128          # 임베딩 차원 수
hidden_dim = 256         # LSTM의 은닉 상태 차원 수
output_dim = 2           # 감정 분류 (0: 부정, 1: 긍정)

# 모델 인스턴스 생성
model = BiLSTM(vocab_size, embed_dim, hidden_dim, output_dim)

print(model)


BiLSTM(
  (embedding): Embedding(45711, 128, padding_idx=0)
  (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)


In [21]:
# 학습 설정정
import torch.nn as nn
import torch.optim as optim

# GPU 사용 설정 (가능한 경우 GPU로 이동)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

# 모델을 장치로 이동
model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 다중 분류 문제에서 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)


사용 중인 장치: cpu


In [22]:
# 모델 학습 함수 정의
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()  # 모델을 학습 모드로 변경
    total_loss = 0

    for sequences, labels in train_loader:
        # 데이터를 GPU로 이동
        sequences, labels = sequences.to(device), labels.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 모델 예측
        outputs = model(sequences)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)  # 평균 손실 반환


In [23]:
# 모델 평가 함수 정의
def evaluate_model(model, criterion, test_loader, device):
    model.eval()  # 평가 모드로 변경
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # 평가 시에는 기울기 계산 X
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

            # 정확도 계산
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total

    return avg_loss, accuracy


In [24]:
# 학습 진행
NUM_EPOCHS = 5  # 에포크 수 (조절 가능)

for epoch in range(NUM_EPOCHS):
    train_loss = train_model(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = evaluate_model(model, criterion, test_loader, device)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"훈련 손실: {train_loss:.4f}")
    print(f"평가 손실: {test_loss:.4f}, 평가 정확도: {test_accuracy:.4f}")


Epoch 1/5
훈련 손실: 0.4161
평가 손실: 0.3579, 평가 정확도: 0.8401
Epoch 2/5
훈련 손실: 0.3041
평가 손실: 0.3417, 평가 정확도: 0.8527
Epoch 3/5
훈련 손실: 0.2289
평가 손실: 0.3532, 평가 정확도: 0.8507
Epoch 4/5
훈련 손실: 0.1492
평가 손실: 0.4172, 평가 정확도: 0.8475
Epoch 5/5
훈련 손실: 0.0845
평가 손실: 0.5421, 평가 정확도: 0.8452


### 모델 추론

In [ ]:
# 텍스트 전처리리
def preprocess_text(text, vocab, stop_words):
    # 특수문자 제거 및 토큰화
    text = re.sub(r"[^가-힣\s]", "", text)  # 한글과 공백을 제외한 문자 제거
    tokens = text.split()  # 공백 기준으로 단어 나누기
    tokens = [token for token in tokens if token not in stop_words]  # 불용어 제거
    return tokens

# 예시 문장
sample_text = "이 영화 정말 재미있어요!"

# 전처리 및 패딩
tokens = preprocess_text(sample_text, vocab, stop_words)
token_ids = [vocab[token] if token in vocab else vocab["<unk>"] for token in tokens]
padded_tokens = token_ids[:30] + [vocab["<pad>"]] * (30 - len(token_ids))  # 길이가 30이 되도록 패딩
input_tensor = torch.tensor([padded_tokens], dtype=torch.long).to(device)  # 모델 입력을 위한 텐서 변환


In [30]:
# 예측
def predict_sentiment(model, input_tensor, device):
    model.eval()  # 평가 모드로 전환
    with torch.no_grad():  # 기울기 계산하지 않음
        output = model(input_tensor)  # 모델 예측
        predicted_class = torch.argmax(output, dim=1).item() 
        
        # 결과 해석 수정 (출력 순서 반전 시)
        predicted_class = 1 - predicted_class  # 0 ↔ 1 변환 # 예측된 클래스 (0 또는 1)
    return predicted_class

# 예측
predicted_class = predict_sentiment(model, input_tensor, device)

# 예측된 감정 출력
print("긍정" if predicted_class == 1 else "부정")


긍정


In [31]:
# 예시 여러개로 테스트
sample_texts = [
    "정말 재미있고 감동적인 영화였습니다.",
    "최악의 영화였어요. 시간 낭비였어요.",
    "평범한 영화였습니다. 그저 그렇네요."
]

for text in sample_texts:
    tokens = preprocess_text(text, vocab, stop_words)
    token_ids = [vocab[token] if token in vocab else vocab["<unk>"] for token in tokens]
    padded_tokens = token_ids[:30] + [vocab["<pad>"]] * (30 - len(token_ids))
    input_tensor = torch.tensor([padded_tokens], dtype=torch.long).to(device)
    
    predicted_class = predict_sentiment(model, input_tensor, device)
    
    print(f"입력 문장: {text}")
    if predicted_class == 0:
        print("예측 결과: 부정적인 감정입니다.")
    else:
        print("예측 결과: 긍정적인 감정입니다.")
    print("-" * 50)


입력 문장: 정말 재미있고 감동적인 영화였습니다.
예측 결과: 긍정적인 감정입니다.
--------------------------------------------------
입력 문장: 최악의 영화였어요. 시간 낭비였어요.
예측 결과: 부정적인 감정입니다.
--------------------------------------------------
입력 문장: 평범한 영화였습니다. 그저 그렇네요.
예측 결과: 긍정적인 감정입니다.
--------------------------------------------------
